In [1]:
import utils

In [2]:
from perceval.backends.core.git import Git

In [3]:
settings = utils.read_config_file('settings.yml')
es = utils.establish_connection(settings['es_host'])

Connection established


In [40]:
MAPPING_GIT = {
    "mappings": {
        "item": {
            "properties": {
                "date": {
                    "type": "date",
                    "format" : "E MMM d HH:mm:ss yyyy Z",
                    "locale" : "US"
                },
                "commit": {"type": "keyword"},
                "author": {"type": "keyword"},
                "domain": {"type": "keyword"},
                "file": {"type": "keyword"},
                "added": {"type": "integer"},
                "removed": {"type": "integer"},
                "repository": {"type": "keyword"}
            }
        }
    }
}

In [41]:
es.indices.delete('git', ignore=[400, 404])
es.indices.create('git', body=MAPPING_GIT)

{'acknowledged': True, 'shards_acknowledged': True}

In [42]:
for repo_url in settings['git']:
    
    repo_name = repo_url.split('/')[-1]
    repo = Git(uri=repo_url, gitpath='/tmp/'+repo_name)
    
    for commit in repo.fetch():
        
        author_name = commit['data']['Author'].split('<')[0][:-1]
        author_domain = commit['data']['Author'].split('@')[-1][:-1]
        
        for file in commit['data']['files']:
            if 'added' not in file.keys() or file['added'] == '-': 
                file['added'] = 0
            if 'removed' not in file.keys() or file['removed'] == '-':
                file['removed'] = 0
            
            summary = {
                'date': commit['data']['AuthorDate'],
                'commit': commit['data']['commit'],
                'author': author_name,
                'domain': author_domain,
                'file': file['file'],
                'added': file['added'],
                'removed': file['removed'],
                'repository': repo_name
            }
            
            es.index(index='git', doc_type='commits', body=summary)